In [18]:
import pandas as pd
import requests
import time
import datetime
from io import StringIO
from bs4 import BeautifulSoup

# 展開報表結果
pd.set_option('display.max_rows', 2000)
pd.set_option('display.max_columns', 2000)
pd.set_option('display.width', 1000)
# 偽瀏覽器
headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}


# 取得前一天收盤價
def getAllCodeWithPrice(df_all_code):
    # 上市代碼網址
    url = 'http://www.twse.com.tw/exchangeReport/STOCK_DAY_ALL?response=open_data'


    # 下載網站內容，並用pandas轉換成 dataframe
    r = requests.get(url, headers=headers)
    r.encoding = 'UTF-8'

    df=pd.read_csv(StringIO(r.content.decode('utf-8')))
    df = df[df['收盤價'].notnull()]
    df['收盤價'] = df['收盤價'].apply(str)
    df_stock = df[['證券代號','收盤價']]

    # 上櫃代碼網址
    url = 'http://www.tpex.org.tw/web/stock/aftertrading/otc_quotes_no1430/stk_wn1430_result.php?l=zh-tw&se=EW&o=data'

    r = requests.get(url, headers=headers)
    r.encoding = 'UTF-8'

    df=pd.read_csv(StringIO(r.content.decode('utf-8')))
    df = df.rename(columns={'代號': '證券代號', '收盤': '收盤價'})
    df = df[df['收盤價'] != '----' ]
    df_otc = df[['證券代號','收盤價']]
    df_all_code_with_closing_price = pd.concat([df_stock,df_otc])
    df_all_code_with_closing_price['收盤價'] = df_all_code_with_closing_price['收盤價'].str.replace(',','')
    df_all_code_with_closing_price = df_all_code_with_closing_price.reset_index(drop=True)

    # 過濾出只包含上市上櫃資料
    df_all_code['收盤價'] = ''
    for index, code in df_all_code.iterrows():
        temp = df_all_code_with_closing_price[df_all_code_with_closing_price['證券代號'] == code['證券代號']]
        if temp.shape[0] > 0:
            code['收盤價'] = temp.iloc[0]['收盤價']
        else:
            code['收盤價'] = '--'
            
    df_all_code = df_all_code.reset_index(drop=True)
    return  df_all_code

def getDividend(year, df_with_price):
    for index, stock in df_with_price.iterrows():
        time.sleep(0.8)
        print('已經處理筆數：' + str(index+1) + ',代碼：' + stock['證券代號'])
        #print(stock['證券代號'] + ',' + stock['收盤價'])

        url = 'https://tw.stock.yahoo.com/d/s/dividend_' + stock['證券代號'] + '.html'
        r = requests.get(url, headers=headers)
        r.encoding = 'big5'
        dfs = pd.read_html(StringIO(r.text), encoding='big5')
        df = pd.concat([df for df in dfs if df.shape[1] == 7])
        df[0] = df[0].str[:3]
        now = datetime.datetime.now()
        # 欄位字串轉數字
        df[0] = pd.to_numeric(df[0], 'coerce')
        df[6] = pd.to_numeric(df[6], 'coerce')
        df = df[~df[0].isnull()]
        df = df[df[1] != '-']
        if df.shape[0] == 0:
            continue
        df = df[[0,6]]
        df = df.rename(columns={0: '年度', 6: '股利'}).groupby(['年度']).sum().sort_index(ascending=False).reset_index()
        df['年度'] = df['年度'].apply(str)
        df['年度'] = df['年度'].str[:3]
        temp = df[df['年度'].str.contains(year)]
        if temp.shape[0] > 0:
            stock[year + '年股利'] = temp.iloc[0]['股利']
    return df_with_price
            
            
#產生出的檔案名稱路徑
exportFilePath = 'C:/Users/Skyluck/Desktop/最新股價.csv'

#讀取代碼範本檔案名稱路徑
importFilePath = 'C:/Users/Skyluck/Desktop/stock_code.xlsx'

#是否要查詢股利  True/False
search_devidend = True 

#要查詢股利之民國年
year = '109'

df_all_code = pd.read_excel(importFilePath)
df_all_code = df_all_code.rename(columns={'代號': '證券代號'})
df_all_code['證券代號'] = df_all_code['證券代號'].apply(str)
df_all_code_with_price = getAllCodeWithPrice(df_all_code)
show_result = []
if search_devidend == False:
    df_all_code_with_price.to_csv(exportFilePath, encoding="utf_8_sig")
    show_result = df_all_code_with_price
else:
    df_all_code_with_price[year + '年股利'] = ''
    df_all_code_with_price_dividend = getDividend(year,df_all_code_with_price)
    df_all_code_with_price_dividend.to_csv(exportFilePath, encoding="utf_8_sig")
    show_result = df_all_code_with_price_dividend

show_result

已經處理筆數：1,代碼：1101
已經處理筆數：2,代碼：1102
已經處理筆數：3,代碼：1103
已經處理筆數：4,代碼：1104
已經處理筆數：5,代碼：1108
已經處理筆數：6,代碼：1109
已經處理筆數：7,代碼：1110
已經處理筆數：8,代碼：1201
已經處理筆數：9,代碼：1203
已經處理筆數：10,代碼：1210
已經處理筆數：11,代碼：1213
已經處理筆數：12,代碼：1215
已經處理筆數：13,代碼：1216
已經處理筆數：14,代碼：1217
已經處理筆數：15,代碼：1218
已經處理筆數：16,代碼：1219
已經處理筆數：17,代碼：1220
已經處理筆數：18,代碼：1225
已經處理筆數：19,代碼：1227
已經處理筆數：20,代碼：1229
已經處理筆數：21,代碼：1231
已經處理筆數：22,代碼：1232
已經處理筆數：23,代碼：1233
已經處理筆數：24,代碼：1234
已經處理筆數：25,代碼：1235
已經處理筆數：26,代碼：1236
已經處理筆數：27,代碼：1256
已經處理筆數：28,代碼：1301
已經處理筆數：29,代碼：1303
已經處理筆數：30,代碼：1304
已經處理筆數：31,代碼：1305
已經處理筆數：32,代碼：1307
已經處理筆數：33,代碼：1308
已經處理筆數：34,代碼：1309
已經處理筆數：35,代碼：1310
已經處理筆數：36,代碼：1312
已經處理筆數：37,代碼：1313
已經處理筆數：38,代碼：1314
已經處理筆數：39,代碼：1315
已經處理筆數：40,代碼：1316
已經處理筆數：41,代碼：1319
已經處理筆數：42,代碼：1321
已經處理筆數：43,代碼：1323
已經處理筆數：44,代碼：1324
已經處理筆數：45,代碼：1325
已經處理筆數：46,代碼：1326
已經處理筆數：47,代碼：1337
已經處理筆數：48,代碼：1338
已經處理筆數：49,代碼：1339
已經處理筆數：50,代碼：1340
已經處理筆數：51,代碼：1341
已經處理筆數：52,代碼：1402
已經處理筆數：53,代碼：1409
已經處理筆數：54,代碼：1410
已經處理筆數：55,代碼：1413
已經處理筆數：56,代碼：1414
已

已經處理筆數：438,代碼：2634
已經處理筆數：439,代碼：2636
已經處理筆數：440,代碼：2637
已經處理筆數：441,代碼：2642
已經處理筆數：442,代碼：2701
已經處理筆數：443,代碼：2702
已經處理筆數：444,代碼：2704
已經處理筆數：445,代碼：2705
已經處理筆數：446,代碼：2706
已經處理筆數：447,代碼：2707
已經處理筆數：448,代碼：2712
已經處理筆數：449,代碼：2722
已經處理筆數：450,代碼：2723
已經處理筆數：451,代碼：2727
已經處理筆數：452,代碼：2731
已經處理筆數：453,代碼：2739
已經處理筆數：454,代碼：2748
已經處理筆數：455,代碼：2801
已經處理筆數：456,代碼：2809
已經處理筆數：457,代碼：2812
已經處理筆數：458,代碼：2816
已經處理筆數：459,代碼：2820
已經處理筆數：460,代碼：2823
已經處理筆數：461,代碼：2832
已經處理筆數：462,代碼：2834
已經處理筆數：463,代碼：2836
已經處理筆數：464,代碼：2838
已經處理筆數：465,代碼：2841
已經處理筆數：466,代碼：2845
已經處理筆數：467,代碼：2849
已經處理筆數：468,代碼：2850
已經處理筆數：469,代碼：2851
已經處理筆數：470,代碼：2852
已經處理筆數：471,代碼：2855
已經處理筆數：472,代碼：2867
已經處理筆數：473,代碼：2880
已經處理筆數：474,代碼：2881
已經處理筆數：475,代碼：2882
已經處理筆數：476,代碼：2883
已經處理筆數：477,代碼：2884
已經處理筆數：478,代碼：2885
已經處理筆數：479,代碼：2886
已經處理筆數：480,代碼：2887
已經處理筆數：481,代碼：2888
已經處理筆數：482,代碼：2889
已經處理筆數：483,代碼：2890
已經處理筆數：484,代碼：2891
已經處理筆數：485,代碼：2892
已經處理筆數：486,代碼：2897
已經處理筆數：487,代碼：2901
已經處理筆數：488,代碼：2903
已經處理筆數：489,代碼：2904
已經處理筆數：490,代

已經處理筆數：870,代碼：8215
已經處理筆數：871,代碼：8222
已經處理筆數：872,代碼：8249
已經處理筆數：873,代碼：8261
已經處理筆數：874,代碼：8271
已經處理筆數：875,代碼：8341
已經處理筆數：876,代碼：8367
已經處理筆數：877,代碼：8374
已經處理筆數：878,代碼：8404
已經處理筆數：879,代碼：8411
已經處理筆數：880,代碼：8422
已經處理筆數：881,代碼：8427
已經處理筆數：882,代碼：8429
已經處理筆數：883,代碼：8442
已經處理筆數：884,代碼：8443
已經處理筆數：885,代碼：8454
已經處理筆數：886,代碼：8462
已經處理筆數：887,代碼：8463
已經處理筆數：888,代碼：8464
已經處理筆數：889,代碼：8466
已經處理筆數：890,代碼：8467
已經處理筆數：891,代碼：8473
已經處理筆數：892,代碼：8478
已經處理筆數：893,代碼：8480
已經處理筆數：894,代碼：8481
已經處理筆數：895,代碼：8482
已經處理筆數：896,代碼：8488
已經處理筆數：897,代碼：8497
已經處理筆數：898,代碼：8499
已經處理筆數：899,代碼：8926
已經處理筆數：900,代碼：8940
已經處理筆數：901,代碼：8996
已經處理筆數：902,代碼：9103
已經處理筆數：903,代碼：9105
已經處理筆數：904,代碼：9136
已經處理筆數：905,代碼：9188
已經處理筆數：906,代碼：9802
已經處理筆數：907,代碼：9902
已經處理筆數：908,代碼：9904
已經處理筆數：909,代碼：9905
已經處理筆數：910,代碼：9906
已經處理筆數：911,代碼：9907
已經處理筆數：912,代碼：9908
已經處理筆數：913,代碼：9910
已經處理筆數：914,代碼：9911
已經處理筆數：915,代碼：9912
已經處理筆數：916,代碼：9914
已經處理筆數：917,代碼：9917
已經處理筆數：918,代碼：9918
已經處理筆數：919,代碼：9919
已經處理筆數：920,代碼：9921
已經處理筆數：921,代碼：9924
已經處理筆數：922,代

,證券代號,收盤價,109年股利
0,1101,42.9,
1,1102,43.3,
2,1103,18.2,
3,1104,22.8,
4,1108,11.9,
5,1109,20.4,
6,1110,17.4,
7,1201,21.4,
8,1203,36.65,
9,1210,46.35,
